### 위경도 좌표를 통해 행정구역 칼럼 추가 코드

##### 데이터 스키마

|id|start time|end time|start lat|start lng|end lat|end lng|?|?|?|start loc|end loc|
|---|---|---|---|---|---|---|---|---|---|---|---|

##### 필요 라이브러리 설치


In [ ]:
#%pip install GDAL
#%pip install pyproj
%pip install Fiona
%pip install Shapely
%pip install geopandas

In [2]:
import os
import geopandas as gpd
import pandas as pd

In [ ]:
# 디렉토리 확인
#os.chdir('../')
os.getcwd()

'c:\\Users\\11\\Desktop\\LCG\\Programming\\project\\wedrive'

In [5]:
# 테스트 데이터
test_data_path=os.getcwd()+'/data/od_uuid/2024/4c17f6c58d754094ba612cef51945447.csv'
location_label_path=os.getcwd()+'/data/행정구역구분/BND_SIGUNGU_PG.shp'

In [6]:
#데이터셋 칼럼명 추가
column_name=['id','start_time','end_time','start_lat','start_lng','end_lat','end_lng','?1','?2','?3']

raw_test_data=pd.read_csv(test_data_path)
data=raw_test_data.values.tolist()
test_data=pd.DataFrame(data, columns=column_name)
loc_data=test_data[['start_lat', 'start_lng']]
loc_data

,start_lat,start_lng
0,37.624019,127.140193
1,37.590619,127.136951
2,37.587170,127.130025
3,37.546898,127.106242
4,37.537902,127.093627
...,...,...
6952,37.535868,127.079745
6953,37.539735,127.078547
6954,37.541160,127.070320
6955,37.529334,127.083380


In [7]:
#행정동 데이터 좌표계 변환
location_label=gpd.read_file(location_label_path, encoding='euc-kr')
location_label.to_crs(epsg=4326, inplace=True)
location_label

,BASE_DATE,SIGUNGU_NM,SIGUNGU_CD,geometry
0,20230701,종로구,11010,"POLYGON ((126.97508 37.63118, 126.97488 37.630..."
1,20230701,중구,11020,"POLYGON ((127.02314 37.57196, 127.02336 37.571..."
2,20230701,용산구,11030,"POLYGON ((126.96917 37.55488, 126.96917 37.554..."
3,20230701,성동구,11040,"POLYGON ((127.04002 37.57299, 127.04041 37.572..."
4,20230701,광진구,11050,"POLYGON ((127.10166 37.57240, 127.10224 37.572..."
...,...,...,...,...
245,20230701,함양군,38580,"POLYGON ((127.69596 35.75773, 127.69647 35.757..."
246,20230701,거창군,38590,"POLYGON ((127.88313 35.90586, 127.88299 35.905..."
247,20230701,합천군,38600,"POLYGON ((128.09861 35.83067, 128.09899 35.830..."
248,20230701,제주시,39010,"MULTIPOLYGON (((126.16791 33.28282, 126.16790 ..."


In [8]:
geometry_Point=gpd.points_from_xy(loc_data['start_lng'], loc_data['start_lat'])
geometry_Point

<GeometryArray>
[<POINT (127 37.6)>, <POINT (127 37.6)>, <POINT (127 37.6)>,
 <POINT (127 37.5)>, <POINT (127 37.5)>, <POINT (127 37.5)>,
 <POINT (127 37.5)>, <POINT (127 37.5)>, <POINT (127 37.5)>,
 <POINT (127 37.5)>,
 ...
 <POINT (127 37.5)>, <POINT (127 37.6)>, <POINT (127 37.6)>,
 <POINT (127 37.5)>, <POINT (127 37.5)>, <POINT (127 37.5)>,
 <POINT (127 37.5)>, <POINT (127 37.5)>, <POINT (127 37.5)>,
 <POINT (127 37.5)>]
Length: 6957, dtype: geometry

In [9]:
from fiona.crs import from_epsg
coulumns = ['start_lat', 'start_lng']
gdf_data = gpd.GeoDataFrame(loc_data[coulumns], geometry=geometry_Point, crs=from_epsg(4326))
gdf_data

,start_lat,start_lng,geometry
0,37.624019,127.140193,POINT (127.14019 37.62402)
1,37.590619,127.136951,POINT (127.13695 37.59062)
2,37.587170,127.130025,POINT (127.13002 37.58717)
3,37.546898,127.106242,POINT (127.10624 37.54690)
4,37.537902,127.093627,POINT (127.09363 37.53790)
...,...,...,...
6952,37.535868,127.079745,POINT (127.07974 37.53587)
6953,37.539735,127.078547,POINT (127.07855 37.53973)
6954,37.541160,127.070320,POINT (127.07032 37.54116)
6955,37.529334,127.083380,POINT (127.08338 37.52933)


In [10]:
from shapely.geometry import Point

gdf_data['geometry'] = gdf_data.apply(lambda row : Point([row['start_lng'], row['start_lat']]), axis=1)
gdf_data['geometry']

0       POINT (127.14019 37.62402)
1       POINT (127.13695 37.59062)
2       POINT (127.13002 37.58717)
3       POINT (127.10624 37.54690)
4       POINT (127.09363 37.53790)
                   ...            
6952    POINT (127.07974 37.53587)
6953    POINT (127.07855 37.53973)
6954    POINT (127.07032 37.54116)
6955    POINT (127.08338 37.52933)
6956    POINT (127.08355 37.53695)
Name: geometry, Length: 6957, dtype: geometry

In [ ]:
df = gpd.sjoin(gdf_data, location_label)
df.drop(columns=['index_right'], axis=1, inplace=True)
df.to_csv(os.getcwd()+'/location_mapping.csv')

#### 실제 위치랑 비교

In [ ]:
import folium

#맵 띄우기
m = folium.Map(location=[df.start_lat[0],df.start_lng[0]],
               zoom_start=7)

path_points = loc_data[['start_lat', 'start_lng']].values.tolist()
for idx, p in enumerate(path_points):
    folium.Circle(p, color='blue',radius = 20, popup=idx).add_to(m)

m.save('location_marker.html')